# CHAPTER 1 NumPy 기본 — 배열과 벡터 연산

## 이 노트북은 무엇인가?
- **NumPy를 '문법'이 아니라 '분석가의 계산 사고'**로 배우는 노트북입니다.
- CSV(표 데이터)를 NumPy 배열(ndarray)로 바꾸고, **shape/ndim/axis/벡터화**를 훈련합니다.

## 사용 데이터
- `data/activity_daily.csv`

## 오늘의 산출물
- `data/user_activity_summary.csv`
- `data/activity_array.npy`


In [1]:
# ============================================
# [공통] 라이브러리 / 경로 / 출력 옵션 세팅
# 이 셀은 모든 챕터 노트북에서 동일하게 사용합니다.
# ============================================

# 1) 수치 계산(NumPy) / 표 데이터(Pandas) 불러오기
import numpy as np
import pandas as pd

# 2) 파일 경로를 운영체제와 무관하게 다루기 위한 Path
from pathlib import Path

# 3) 현재 노트북이 실행되는 폴더를 기준(BASE)으로 데이터 폴더(DATA) 지정
BASE = Path(".").resolve()          # 현재 작업 폴더(절대경로)
DATA = BASE / "data"                # data 폴더 경로 

# 4) Pandas 출력 옵션(교육용): 너무 길게 출력되지 않도록 적당히 제한
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 140)

# 5) 확인 출력
print("BASE:", BASE)
print("DATA exists:", DATA.exists()) # 우리가 존재하는 위치값 확인 


BASE: C:\Users\KDA\2026_KDA3
DATA exists: True


## 1.1 ndarray 구조 이해 (shape, ndim, dtype)
### 핵심 질문
- 행(row)은 무엇인가? (관측치)
- 열(column)은 무엇인가? (변수)


In [2]:
# [이 셀은 무엇을 하는가?]
# - CSV를 불러와서 데이터의 '구조(행/열/기간/유니크값)'를 확인합니다.

df = pd.read_csv(DATA / "activity_daily.csv")          # CSV → DataFrame
display(df.head())                                     # 상위 5행 미리보기

print("df.shape (행,열):", df.shape)                   # 데이터 크기
print("고유 사용자 수:", df["user_id"].nunique())      # 유니크 사용자 수 -> 행별로 사용자 수 -> 사용자 수를 받음 
print("기간:", df["date"].min(), "~", df["date"].max())# 기간 -> 최댓값 최솟값을 정해서 -> 1년의값/분기값을 정할 수 있다 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KDA\\2026_KDA3\\data\\activity_daily.csv'

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 수치형 컬럼만 골라 ndarray로 변환하고, ndarray 메타정보(shape/ndim/dtype)를 확인합니다.

num_df = df[["steps", "active_minutes", "calories"]]   # 수치형 컬럼 선택
arr = num_df.to_numpy()                                 # DataFrame → ndarray

print("type(arr):", type(arr))                          # ndarray 타입 확인
print("arr[:3]:\n", arr[:3])                            # 앞 3행 출력 -> 어디까지 기준까지 출력 해줘야하는지에 따라 작성 

print("shape:", arr.shape)                              # (관측치 수, 변수 수)  -> user값, 변수 값 
print("ndim :", arr.ndim)                               # 차원 수 -> 1차원/2차원/3차원... 
print("size :", arr.size)                               # 전체 원소 수 -> 
print("dtype:", arr.dtype)                              # 자료형 -> 데이터 타입 반환 받기 / astype -> 타입 변환가능 

## 4.2 axis 사고 (축을 기준으로 계산하기)
- axis=0: 열(변수) 기준
- axis=1: 행(관측치) 기준


In [ ]:
# [이 셀은 무엇을 하는가?]
# - axis=0/1 차이를 직접 계산으로 확인하고, 결과를 해석 가능한 형태로 다시 DataFrame에 붙입니다.

col_mean = arr.mean(axis=0)                              # 지표별 평균
col_sum  = arr.sum(axis=0)                               # 지표별 합
print("col_mean [steps, active, cal]:", col_mean)
print("col_sum  [steps, active, cal]:", col_sum)

row_sum = arr.sum(axis=1)                                # 기록별 합(파생지표)
df2 = df.copy()                                          # 원본 보존 
df2["total_activity"] = row_sum                          # 파생컬럼 추가

top5 = df2.sort_values("total_activity", ascending=False).head(5) #  ascending=False -> '내림차순'으로 정렬되어 출력 
display(top5[["date","user_id","total_activity","steps","active_minutes","calories"]])

## 1.3 ufunc (np.log, np.sqrt, np.where)
- for문 없이 배열 전체 연산
- 분포 안정화(로그)
- 조건 기반 파생지표


In [ ]:
# [이 셀은 무엇을 하는가?]
# - ufunc를 이용해 로그 변환/정규화/조건 파생지표를 만듭니다.

steps = arr[:, 0]                                        # steps 1차원 추출 # 행과 열을 기준으로 만듦 
log_steps = np.log(steps + 1)                             # log(0) 방지로 +1
steps_norm = steps / steps.max()                          # 0~1 정규화

is_high = np.where(steps >= 8000, 1, 0)                   # where -> 조건: 8000 이상이면 1

print("steps[:5]     :", steps[:5])
print("log_steps[:5] :", log_steps[:5])
print("norm_steps[:5]:", steps_norm[:5])
print("is_high[:10]  :", is_high[:10])

## 1.4 Boolean mask로 필터링(질문 만들기)
- 조건을 mask로 만들고
- 조건 만족 데이터만 뽑고
- 비율(%)을 계산


In [ ]:
# [이 셀은 무엇을 하는가?]
# - steps>=8000 조건을 mask로 만들고, 해당 조건의 비율과 샘플을 확인합니다.

mask = steps >= 8000                                     # Boolean mask 생성
ratio = mask.mean()                                      # True 비율(평균)

print("고활동(steps>=8000) 비율:", ratio)
high_df = df2.loc[mask].copy()                            # 조건 만족 행만 필터링
display(high_df.head())

## 1.5 NumPy 저장/로드 (.npy)
- 중간 결과 캐싱
- 재계산 방지


In [ ]:
# [이 셀은 무엇을 하는가?]
# - ndarray를 .npy로 저장하고 다시 불러와 동일성 검증합니다.

npy_path = DATA / "activity_array.npy"                    # 저장 경로
np.save(npy_path, arr)                                    # 저장
loaded = np.load(npy_path)                                # 로드

print("saved:", npy_path)
print("loaded shape:", loaded.shape)
print("same:", np.allclose(arr, loaded))                  # 동일성 확인

## 1.6 상관관계(변수 관계 요약) -> 중요
- corrcoef로 상관계수 계산


In [ ]:
# [이 셀은 무엇을 하는가?]
# - steps/active_minutes/calories 간 상관계수를 계산해 관계를 요약합니다.

corr = np.corrcoef(arr, rowvar=False)                     # 변수(열) 기준 상관계수 ->corrcoef
corr_df = pd.DataFrame(
    corr,
    index=["steps","active_minutes","calories"],
    columns=["steps","active_minutes","calories"]
)
display(corr_df.round(3))

## 1.7 누적 합(cumsum)으로 성장/누적 사고 훈련

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 특정 사용자(U001)의 steps 누적합을 만들어 '성장 곡선'을 만듭니다.

df2["date"] = pd.to_datetime(df2["date"])                 # 날짜형 변환
u_df = df2[df2["user_id"] == "U001"].sort_values("date").copy()
u_df["cum_steps"] = u_df["steps"].cumsum()                # 누적합 -> cumsum

display(u_df[["date","user_id","steps","cum_steps"]].head(10))

## 1.8 산출물: 사용자 요약표 저장

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 사용자 단위 요약 리포트를 만들고 CSV로 저장합니다(다음 챕터에서 재사용).

summary = df2.groupby("user_id").agg(
    days=("date","nunique"),
    avg_steps=("steps","mean"),
    std_steps=("steps","std"),
    max_steps=("steps","max"),
    high_days=("steps", lambda x: (x >= 8000).sum()),
    avg_calories=("calories","mean")
).sort_values("avg_steps", ascending=False)

display(summary)

out_csv = DATA / "user_activity_summary.csv"
summary.to_csv(out_csv)
print("saved:", out_csv)